# Gender, Mental Illness, and Crime in the United States, 2004 Analysis

In [2]:
!pip install -r requirements.txt

## Sprawdzenie czy dane są dostępne

In [3]:
import os

file_path = os.path.join(os.getcwd(), 'ICPSR_27521-V1', 'ICPSR_27521', 'DS0001', '27521-0001-Data.tsv')
if not os.path.exists(file_path):
    raise FileNotFoundError(f'{file_path} not found\n Check the path to the data')

## Wczytanie danych

In [18]:
import pandas as pd
df = pd.read_csv(file_path, sep='\t', low_memory=False)

Sprawdzenie wymiarów zbioru danych

In [22]:
print(df.shape[0], 'rows')
print(df.shape[1], 'columns')

55602 rows
3011 columns


Przetestowałem wykorzystanie dataframe z biblioteki `dask`, ale większość poleceń poza wczytywaniem była znacząco dłuższa (wykorzystanie `.compute()` dla wczytanego dataframe zajmowało 2-3x więcej czasu niż wczytanie przy pomocy `pandas`)

## Wstępna analiza

Po przeczytaniu następujących artykułów:
- Silverstein, Brett; Levin, Eitan, "Differences in the developmental
patterns of depression with and without additional somatic symptoms."
Psychiatry Research. Dec 2014, 220, (1-2), 254 - 257. DOI:
10.1016/j.psychres.2014.07.054
- Thompson, Melissa, "Gender, Mental Illness, and Crime." Final Technical
Report. NCJ 224028, Sep 2008. 2007-IJ-CX-0004

oraz przejrzeniu Codebook, do wstępnej analizy wybrane zostały następujące cechy (w nawiasie % brakujących danych):
- rodzaj depresji w ostatnim roku:
  - utworzona zostanie nowa kolumna `DEPRESSIONTYPELASTYR`, która przyjmie następujące wartości:
    - `0`: brak depresji,
    - `1`: 'czysta' depresja,
    - `2`: depresja somatyczna;
  - wykorzystane zostaną kolumny:
    - czy wystąpił silny epizod depresyjny w ostatnim roku `MDEYR` (0.8%),
    - 
- płeć: `IRSEX` (0%),
- wiek: `CATAG7` (0%),
- współczynnik uzależnienia od narkotyków: `ANYINDEX` ()
- kiedy ostatnio brał jakiekolwiek nielegalne narkotyki `ANYILLICITRC` (0%)
- czy popełnił przestępstwo: `ANYCRIME` (0.1%),
- czy kiedykolwiek został aresztowany: `CRIMEHIST` (0.3%),
- czy kiedykolwiek użył środków przeciwbólowych do celów niemedycznych `ANLEVER` (0.3%)
- czy pracuje lub chodzi do szkoły `WORKSCHOOL` (0%)